# Compression d'images en couleurs

Dans cette partie, nous allons utiliser une sous partie du dataset ImageNet: https://www.kaggle.com/datasets/deeptrial/miniimagenet/data

In [1]:
cd ../src

c:\Users\fredl\Documents\Cours\M1 Androide\S2\ML\Projet\src


C:\Users\fredl\AppData\Roaming\Python\Python313\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import kagglehub

path = kagglehub.dataset_download("deeptrial/miniimagenet")

print("Path to dataset files:", path)

c:\Users\fredl\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: C:\Users\fredl\.cache\kagglehub\datasets\deeptrial\miniimagenet\versions\2


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
from sklearn.manifold import TSNE
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from activation_func import *
from module import *
from loss import *
from optimizers import *
from mltools import *

In [4]:
image_files = []
for root, dirs, files in os.walk(os.path.join(path, "ImageNet-Mini", "images")):
    for file in files:
        if file.endswith(('.png', '.jpg', '.jpeg', '.JPEG')):
            image_files.append(os.path.join(root, file))

images = [Image.open(img_file) for img_file in image_files]

print(f"Loaded {len(images)} images from the dataset.")

Loaded 3923 images from the dataset.


In [5]:
fixed_size = (224, 224)
resized_images = [img.resize(fixed_size) for img in images]

print(f"Resized {len(resized_images)} images to {fixed_size}.")

Resized 3923 images to (224, 224).


### Convertion des images en RGB, HSV et OKLCH

In [ ]:
from skimage.color import rgb2hsv, rgb2lab

def preprocess_images(images):
    rgb_images = [np.array(img) / 255.0 for img in images]  # Normalisation entre 0 et 1
    # hsv_images = [rgb2hsv(img) for img in rgb_images if img.shape[-1] == 3]
    # oklch_images = [rgb2lab(img) for img in rgb_images]  # TODO: LAB est utilisé comme placeholder pour OKLCH, à remplacer
    # return rgb_images, hsv_images, oklch_images
    return rgb_images

# Normaliser les valeurs entre -1 et 1
def normalize_images(images):
    return [(img * 2 - 1) for img in images]

In [14]:
# Préparer les données
# rgb_images, hsv_images, oklch_images = preprocess_images(resized_images)
rgb_images = preprocess_images(resized_images)
# hsv_images = normalize_images(hsv_images)
# oklch_images = normalize_images(oklch_images)

# Aplatir les images pour les passer dans l'autoencodeur
rgb_flattened = [img.flatten() for img in rgb_images]
# hsv_flattened = [img.flatten() for img in hsv_images]
# oklch_flattened = [img.flatten() for img in oklch_images]

### Expérience 1 : Trois autoencodeurs distincts pour R, G et B

In [31]:
input_dim_channel = fixed_size[0] * fixed_size[1]

autoencoder_r = AutoEncoder([input_dim_channel, 8192, 4096, 1024], [1024, 4096, 8192, input_dim_channel])
autoencoder_g = AutoEncoder([input_dim_channel, 8192, 4096, 1024], [1024, 4096, 8192, input_dim_channel])
autoencoder_b = AutoEncoder([input_dim_channel, 8192, 4096, 1024], [1024, 4096, 8192, input_dim_channel])

r_channel = [img[:, :, 0].flatten() for img in rgb_images if img.ndim == 3]
g_channel = [img[:, :, 1].flatten() for img in rgb_images if img.ndim == 3]
b_channel = [img[:, :, 2].flatten() for img in rgb_images if img.ndim == 3]

In [32]:
loss_fn = MSELoss()
learning_rate = 1e-3
epochs = 10
batch_size = 64

print("Training R Channel Autoencoder...")
optimizer_r = Optim(autoencoder_r, loss_fn, learning_rate)
losses_r = optimizer_r.SGD(r_channel, r_channel, batch_size=batch_size, num_iterations=epochs)

print("Training G Channel Autoencoder...")
optimizer_g = Optim(autoencoder_g, loss_fn, learning_rate)
losses_g = optimizer_g.SGD(g_channel, g_channel, batch_size=batch_size, num_iterations=epochs)

print("Training B Channel Autoencoder...")
optimizer_b = Optim(autoencoder_b, loss_fn, learning_rate)
losses_b = optimizer_b.SGD(b_channel, b_channel, batch_size=batch_size, num_iterations=epochs)

Training R Channel Autoencoder...


  0%|          | 0/10 [00:00<?, ?it/s]


TypeError: only integer scalar arrays can be converted to a scalar index

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(losses_r, label='R Channel Loss')
plt.plot(losses_g, label='G Channel Loss')
plt.plot(losses_b, label='B Channel Loss')
plt.title('Training Losses for Each Channel Autoencoder')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

def plot_reconstructed_images(original, reconstructed, channel_name):
    plt.figure(figsize=(12, 6))
    for i in range(5):
        plt.subplot(2, 5, i + 1)
        plt.imshow(original[i].reshape(fixed_size[0], fixed_size[1]), cmap='gray')
        plt.title(f'Original {channel_name}')
        plt.axis('off')

        plt.subplot(2, 5, i + 6)
        plt.imshow(reconstructed[i].reshape(fixed_size[0], fixed_size[1]), cmap='gray')
        plt.title(f'Reconstructed {channel_name}')
        plt.axis('off')
    plt.show()

reconstructed_r = [autoencoder_r.forward(img) for img in r_channel]
reconstructed_g = [autoencoder_g.forward(img) for img in g_channel]
reconstructed_b = [autoencoder_b.forward(img) for img in b_channel]

plot_reconstructed_images(r_channel, reconstructed_r, 'R Channel')
plot_reconstructed_images(g_channel, reconstructed_g, 'G Channel')
plot_reconstructed_images(b_channel, reconstructed_b, 'B Channel')